In [ ]:
import os
import time
from datetime import datetime
from flask import Flask, request, jsonify
from flask_cors import CORS
from groq import Groq
from dotenv import load_dotenv

load_dotenv()

app = Flask(__name__)
CORS(app)

groq_client = Groq(api_key="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")

user_sessions = {}
generated_content = {}

class SEOContentGenerator:
    def __init__(self, groq_client):
        self.client = groq_client
        self.model = "llama-3.3-70b-versatile"

    def generate_keywords(self, seed_keyword):
        """
        Generate 5 related keywords based on seed keyword
        """
        prompt = f"""
        As an SEO expert, generate 5 highly relevant and SEO-friendly keywords related to '{seed_keyword}'.
        The keywords should be:
        - Semantically related to the seed keyword
        - Good for SEO targeting
        - Include both short-tail and long-tail variations
        - Commercially viable
        
        Return only the keywords in a simple list format, one per line, without numbering or bullet points.
        """

        try:
            response = self.client.chat.completions.create(
                messages=[
                    {"role": "system", "content": "You are an SEO expert specializing in keyword research."},
                    {"role": "user", "content": prompt}
                ],
                model=self.model,
                temperature=0.7,
                max_tokens=200
            )

            keywords_text = response.choices[0].message.content.strip()
            keywords = [kw.strip()
                        for kw in keywords_text.split('\n') if kw.strip()]
            return keywords[:5]

        except Exception as e:
            print(f"Error generating keywords: {e}")
            return [f"{seed_keyword} tips", f"best {seed_keyword}", f"{seed_keyword} guide", f"{seed_keyword} benefits", f"{seed_keyword} solutions"]

    def generate_titles(self, keyword, tone="professional"):
        """
        Generate 3 SEO-optimized titles for the selected keyword
        """
        prompt = f"""
        Create 3 SEO-optimized, engaging titles for the keyword '{keyword}' with a {tone} tone.
        The titles should be:
        - Between 50-60 characters for optimal SEO
        - Include the target keyword naturally
        - Be click-worthy and engaging
        - Follow SEO best practices
        - Be unique and varied in approach
        
        Return only the titles, one per line, without numbering or bullet points.
        """

        try:
            response = self.client.chat.completions.create(
                messages=[
                    {"role": "system", "content": "You are an expert SEO copywriter specializing in creating compelling, SEO-optimized titles."},
                    {"role": "user", "content": prompt}
                ],
                model=self.model,
                temperature=0.8,
                max_tokens=300
            )

            titles_text = response.choices[0].message.content.strip()
            titles = [title.strip()
                      for title in titles_text.split('\n') if title.strip()]
            return titles[:3]

        except Exception as e:
            print(f"Error generating titles: {e}")
            return [
                f"The Ultimate Guide to {keyword}",
                f"How to Master {keyword}: Expert Tips",
                f"{keyword}: Everything You Need to Know"
            ]

    def generate_topics(self, title, keyword):
        """
        Generate 2 topic ideas or outline based on the selected title
        """
        prompt = f"""
        Based on the title '{title}' and keyword '{keyword}', create 2 comprehensive topic outlines.
        Each topic should include:
        - A brief topic description (1-2 sentences)
        - 3-4 main points or sections to cover
        - SEO considerations
        
        Format as:
        Topic 1: [Description]
        - Point 1
        - Point 2
        - Point 3
        
        Topic 2: [Description]
        - Point 1
        - Point 2
        - Point 3
        """

        try:
            response = self.client.chat.completions.create(
                messages=[
                    {"role": "system", "content": "You are a content strategist expert in creating detailed topic outlines for SEO content."},
                    {"role": "user", "content": prompt}
                ],
                model=self.model,
                temperature=0.7,
                max_tokens=500
            )

            return response.choices[0].message.content.strip()

        except Exception as e:
            print(f"Error generating topics: {e}")
            return f"Topic 1: Complete guide to {keyword}\n- Introduction and basics\n- Key strategies\n- Best practices\n\nTopic 2: Advanced {keyword} techniques\n- Expert tips\n- Common mistakes\n- Implementation guide"

    def generate_content(self, keyword, title, topic_outline, content_type="blog_intro"):
        """
        Generate SEO-optimized content (150-250 words)
        """
        if content_type == "meta_description":
            word_count = "150-160 characters"
            content_instruction = "Create a compelling meta description"
        else:
            word_count = "150-250 words"
            content_instruction = "Write an engaging blog post introduction"

        prompt = f"""
        {content_instruction} for:
        - Keyword: {keyword}
        - Title: {title}
        - Topic Outline: {topic_outline}
        
        Requirements:
        - Length: {word_count}
        - Include the target keyword naturally 2-3 times
        - SEO-optimized and engaging
        - Professional tone
        - Include a call-to-action if appropriate
        - Make it compelling and informative
        
        Return only the content without any additional formatting or labels.
        """

        try:
            response = self.client.chat.completions.create(
                messages=[
                    {"role": "system", "content": "You are an expert SEO content writer specializing in creating high-quality, optimized content."},
                    {"role": "user", "content": prompt}
                ],
                model=self.model,
                temperature=0.7,
                max_tokens=400
            )

            return response.choices[0].message.content.strip()

        except Exception as e:
            print(f"Error generating content: {e}")
            return f"Discover everything you need to know about {keyword}. This comprehensive guide covers the essential aspects of {keyword}, providing you with expert insights and practical tips. Whether you're a beginner or looking to enhance your knowledge, our detailed analysis will help you understand the key concepts and best practices. Learn how to effectively implement {keyword} strategies and achieve better results. Start your journey today and unlock the full potential of {keyword} with our proven methods and expert guidance."

    def calculate_seo_score(self, content, keyword):
        """
        Calculate basic SEO score based on keyword presence and content quality
        """
        score = 0
        factors = []

        keyword_lower = keyword.lower()
        content_lower = content.lower()
        keyword_count = content_lower.count(keyword_lower)

        if keyword_count >= 2:
            score += 30
            factors.append("Good keyword frequency")
        elif keyword_count == 1:
            score += 15
            factors.append("Moderate keyword frequency")
        else:
            factors.append("Low keyword frequency")

        word_count = len(content.split())
        if 150 <= word_count <= 300:
            score += 25
            factors.append("Optimal content length")
        elif word_count > 100:
            score += 15
            factors.append("Acceptable content length")
        else:
            factors.append("Content too short")

        sentences = len([s for s in content.split('.') if s.strip()])
        if sentences > 0:
            avg_words_per_sentence = word_count / sentences
            if 10 <= avg_words_per_sentence <= 20:
                score += 20
                factors.append("Good readability")
            else:
                score += 10
                factors.append("Moderate readability")

        if any(char in content for char in ['.', ',', '!', '?']):
            score += 15
            factors.append("Good punctuation")

        if content and content[0].isupper():
            score += 10
            factors.append("Proper capitalization")

        return min(score, 100), factors


seo_generator = SEOContentGenerator(groq_client)


@app.route('/api/health', methods=['GET'])
def health_check():
    """Health check endpoint"""
    return jsonify({
        'status': 'healthy',
        'timestamp': datetime.now().isoformat(),
        'service': 'SEO Scientist AI Content Writer'
    })


@app.route('/api/keywords', methods=['POST'])
def generate_keywords():
    """Generate keywords based on seed keyword"""
    try:
        data = request.get_json()
        seed_keyword = data.get('seed_keyword', '').strip()

        if not seed_keyword:
            return jsonify({'error': 'Seed keyword is required'}), 400

        keywords = seo_generator.generate_keywords(seed_keyword)

        session_id = f"session_{int(time.time())}"
        user_sessions[session_id] = {
            'seed_keyword': seed_keyword,
            'keywords': keywords,
            'timestamp': datetime.now().isoformat()
        }

        return jsonify({
            'session_id': session_id,
            'seed_keyword': seed_keyword,
            'keywords': keywords,
            'status': 'success'
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500


@app.route('/api/titles', methods=['POST'])
def generate_titles():
    """Generate SEO-optimized titles for selected keyword"""
    try:
        data = request.get_json()
        keyword = data.get('keyword', '').strip()
        session_id = data.get('session_id', '')
        tone = data.get('tone', 'professional')

        if not keyword:
            return jsonify({'error': 'Keyword is required'}), 400

        titles = seo_generator.generate_titles(keyword, tone)

        if session_id in user_sessions:
            user_sessions[session_id]['selected_keyword'] = keyword
            user_sessions[session_id]['titles'] = titles
            user_sessions[session_id]['tone'] = tone

        return jsonify({
            'session_id': session_id,
            'keyword': keyword,
            'titles': titles,
            'tone': tone,
            'status': 'success'
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500


@app.route('/api/topics', methods=['POST'])
def generate_topics():
    """Generate topic outlines based on selected title"""
    try:
        data = request.get_json()
        title = data.get('title', '').strip()
        keyword = data.get('keyword', '').strip()
        session_id = data.get('session_id', '')

        if not title or not keyword:
            return jsonify({'error': 'Title and keyword are required'}), 400

        topics = seo_generator.generate_topics(title, keyword)

        if session_id in user_sessions:
            user_sessions[session_id]['selected_title'] = title
            user_sessions[session_id]['topics'] = topics

        return jsonify({
            'session_id': session_id,
            'title': title,
            'keyword': keyword,
            'topics': topics,
            'status': 'success'
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500


@app.route('/api/content', methods=['POST'])
def generate_content():
    """Generate SEO-optimized content based on inputs"""
    try:
        data = request.get_json()
        keyword = data.get('keyword', '').strip()
        title = data.get('title', '').strip()
        topic_outline = data.get('topic_outline', '').strip()
        content_type = data.get('content_type', 'blog_intro')
        session_id = data.get('session_id', '')

        if not all([keyword, title, topic_outline]):
            return jsonify({'error': 'Keyword, title, and topic outline are required'}), 400

        content = seo_generator.generate_content(
            keyword, title, topic_outline, content_type)

        seo_score, seo_factors = seo_generator.calculate_seo_score(
            content, keyword)

        content_id = f"content_{int(time.time())}"

        generated_content[content_id] = {
            'keyword': keyword,
            'title': title,
            'topic_outline': topic_outline,
            'content': content,
            'content_type': content_type,
            'seo_score': seo_score,
            'seo_factors': seo_factors,
            'word_count': len(content.split()),
            'character_count': len(content),
            'timestamp': datetime.now().isoformat()
        }

        if session_id in user_sessions:
            user_sessions[session_id]['generated_content_id'] = content_id
            user_sessions[session_id]['final_content'] = content

        return jsonify({
            'session_id': session_id,
            'content_id': content_id,
            'content': content,
            'seo_score': seo_score,
            'seo_factors': seo_factors,
            'word_count': len(content.split()),
            'character_count': len(content),
            'content_type': content_type,
            'status': 'success'
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500


@app.route('/api/export', methods=['POST'])
def export_content():
    """Export generated content in various formats"""
    try:
        data = request.get_json()
        content_id = data.get('content_id')
        export_format = data.get('format', 'text')

        if not content_id or content_id not in generated_content:
            return jsonify({'error': 'Invalid content ID'}), 400

        content_data = generated_content[content_id]

        if export_format == 'json':
            return jsonify({
                'export_data': content_data,
                'format': 'json',
                'status': 'success'
            })
        else:

            text_export = f"""SEO Content Export
==================
Keyword: {content_data['keyword']}
Title: {content_data['title']}
Content Type: {content_data['content_type']}
SEO Score: {content_data['seo_score']}/100
Word Count: {content_data['word_count']}
Generated: {content_data['timestamp']}

Content:
--------
{content_data['content']}

Topic Outline:
--------------
{content_data['topic_outline']}

SEO Factors:
------------
{chr(10).join('- ' + factor for factor in content_data['seo_factors'])}
"""

            return jsonify({
                'export_data': text_export,
                'format': 'text',
                'status': 'success'
            })

    except Exception as e:
        return jsonify({'error': str(e)}), 500


@app.route('/api/session/<session_id>', methods=['GET'])
def get_session(session_id):
    """Get session data"""
    if session_id not in user_sessions:
        return jsonify({'error': 'Session not found'}), 404

    return jsonify({
        'session_data': user_sessions[session_id],
        'status': 'success'
    })


@app.route('/api/dashboard', methods=['GET'])
def dashboard():
    """Get dashboard data with recent activities"""
    recent_sessions = list(user_sessions.values())[-10:]
    recent_content = list(generated_content.values())[-10:]

    stats = {
        'total_sessions': len(user_sessions),
        'total_content_generated': len(generated_content),
        'avg_seo_score': sum(content.get('seo_score', 0) for content in generated_content.values()) / max(len(generated_content), 1)
    }

    return jsonify({
        'stats': stats,
        'recent_sessions': recent_sessions,
        'recent_content': recent_content,
        'status': 'success'
    })


@app.route('/', methods=['GET'])
def home():
    """Home endpoint with API documentation"""
    return jsonify({
        'service': 'SEO Scientist AI Content Writer API',
        'version': '1.0.0',
        'endpoints': {
            'POST /api/keywords': 'Generate keywords from seed keyword',
            'POST /api/titles': 'Generate SEO titles from keyword',
            'POST /api/topics': 'Generate topic outlines from title',
            'POST /api/content': 'Generate SEO content',
            'POST /api/export': 'Export generated content',
            'GET /api/session/<id>': 'Get session data',
            'GET /api/dashboard': 'Get dashboard statistics',
            'GET /api/health': 'Health check'
        },
        'status': 'running'
    })


def test_seo_generator():
    """Test the SEO generator with sample data"""
    print("Testing SEO Content Generator...")
    print("=" * 50)

    print("\n1. Testing Keyword Generation:")
    keywords = seo_generator.generate_keywords("digital marketing")
    print(f"Generated keywords: {keywords}")

    print("\n2. Testing Title Generation:")
    titles = seo_generator.generate_titles(
        keywords[0] if keywords else "digital marketing")
    print(f"Generated titles: {titles}")

    print("\n3. Testing Topic Generation:")
    topics = seo_generator.generate_topics(
        titles[0] if titles else "Digital Marketing Guide", "digital marketing")
    print(f"Generated topics:\n{topics}")

    print("\n4. Testing Content Generation:")
    content = seo_generator.generate_content(
        "digital marketing", titles[0] if titles else "Digital Marketing Guide", topics)
    print(f"Generated content:\n{content}")

    print("\n5. Testing SEO Scoring:")
    score, factors = seo_generator.calculate_seo_score(
        content, "digital marketing")
    print(f"SEO Score: {score}/100")
    print(f"SEO Factors: {factors}")


if __name__ == '__main__':

    if not os.getenv('GROQ_API_KEY'):
        print("ERROR: GROQ_API_KEY not found in environment variables!")
        print("Please add your Groq API key to the .env file")
        exit(1)

    print("SEO Scientist AI Content Writer Backend")
    print("=====================================")
    print(f"Starting server on http://localhost:5000")
    print(f"Using Groq API with model: {seo_generator.model}")
    print("Press Ctrl+C to stop the server")
    print("\nAvailable endpoints:")
    print("- POST /api/keywords - Generate keywords")
    print("- POST /api/titles - Generate titles")
    print("- POST /api/topics - Generate topics")
    print("- POST /api/content - Generate content")
    print("- POST /api/export - Export content")
    print("- GET /api/dashboard - Dashboard data")
    print("- GET /api/health - Health check")
    print("-" * 50)

    app.run(host='0.0.0.0', port=5000, debug=True,use_reloader=False)

SEO Scientist AI Content Writer Backend
Starting server on http://localhost:5000
Using Groq API with model: llama-3.3-70b-versatile
Press Ctrl+C to stop the server

Available endpoints:
- POST /api/keywords - Generate keywords
- POST /api/titles - Generate titles
- POST /api/topics - Generate topics
- POST /api/content - Generate content
- POST /api/export - Export content
- GET /api/dashboard - Dashboard data
- GET /api/health - Health check
--------------------------------------------------
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.102:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Jun/2025 08:07:11] "OPTIONS /api/keywords HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2025 08:07:12] "POST /api/keywords HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2025 08:07:17] "OPTIONS /api/titles HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2025 08:07:17] "POST /api/titles HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2025 08:07:21] "OPTIONS /api/topics HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2025 08:07:23] "POST /api/topics HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2025 08:07:31] "OPTIONS /api/content HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2025 08:07:32] "POST /api/content HTTP/1.1" 200 -
